In [1]:
from qiskit.optimization import QuadraticProgram

# Method 1: Creating QuadraticProgram from DocPlex Model

In [19]:
# Make a Docplex model
from docplex.mp.model import Model
dcpxMdl = Model('AMC Docplex Model')
x = dcpxMdl.binary_var('x')
y = dcpxMdl.integer_var(lb=-1, ub=5, name='y')
dcpxMdl.minimize(2*x + 7 * y)
dcpxMdl.add_constraint(x - y == 3)
dcpxMdl.add_constraint((x + y) * (x - y) <= 1)
print(dcpxMdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Docplex Model

Minimize
 obj: 2 x + 7 y
Subject To
 c1: x - y = 3
 qc1: [ x^2 - y^2 ] <= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5

Binaries
 x

Generals
 y
End



In [20]:
# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(dcpxMdl)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Docplex Model

Minimize
 obj: 2 x + 7 y
Subject To
 c0: x - y = 3
 q0: [ x^2 - y^2 ] <= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5

Binaries
 x

Generals
 y
End



# Method 2: Directly constructing a QuadraticProgram

In [ ]:
#We start from an empty model.
mod = QuadraticProgram('my problem')
print(mod.export_as_lp_string())

The QuadraticProgram supports three types of variables: 
- Binary variable 
- Integer variable 
- Continuous variable

For a variable, one can specify names, types, lower bounds and upper bounds.

During display as LP format, Binaries denotes binary variables and Generals denotes integer variables. 
If variables are not included in either Binaries or Generals, they are continuous variables with default<br> 
lower bound = 0 and <br>
upper bound = infinity. 

<i>Note that you cannot use ‘e’ or ‘E’ as the first character of names due to the specification of LP format.</i>


In [22]:
# Add variables
mod.binary_var(name='x')
mod.integer_var(name='y', lowerbound=-1, upperbound=5)
mod.continuous_var(name='z', lowerbound=-1, upperbound=5)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj:
Subject To

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



Now we set the objective function by invoking QuadraticProgram.minimize or QuadraticProgram.maximize. 
We may add a constant term as well as linear and quadratic objective function 
by specifying linear and quadratic terms with either list, matrix or dictionary.

Note that in the LP format the quadratic part has to be scaled by a factor 1/2. 

Thus, when printing as LP format, the quadratic part is first multiplied by 2 and then divided by 2 again.



In [24]:
# Add objective function using dictionaries
mod.minimize(constant=3, linear={'x': 1}, quadratic={('x', 'y'): 2, ('z', 'z'): -1})
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x + [ 4 x*y - 2 z^2 ]/2 + 3
Subject To

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



In [27]:
# Add objective function using lists/arrays
mod.minimize(constant=3, linear=[1,0], quadratic=[[0,1],[1,-1]])
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x + [ 4 x*y - 2 y^2 ]/2 + 3
Subject To

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



In [28]:
#Access the constant, the linear term, and the quadratic term from Quadratic.objective
print('constant:\t\t\t', mod.objective.constant)
print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())

#As for linear and quadratic terms, we get a dense matrix (to_array), a sparse matrix (coefficients), 
#and a dictionary (to_dict). 
#For dictionaries, we may specify whether to use variable indices or names as keys. 
#Note that the quadratic terms are stored in a compressed way, 
#e.g., {('x', 'y'): 1, ('y', 'x'): 2} is stored as {('x', 'y'): 3}. 
#You can get the quadratic term as a symmetric matrix by calling to_array(symmetric=True) or 
#to_dict(symmetric=True). 
#If you call to_dict(name=True), you can get a dictionary whose keys are pairs of variable names.
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')
print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)

constant:			 3
linear dict:			 {0: 1}
linear array:			 [1 0]
linear array as sparse matrix:
   (0, 0)	1 

quadratic dict w/ index:	 {(0, 1): 2, (1, 1): -1}
quadratic dict w/ name:		 {('x', 'y'): 2, ('y', 'y'): -1}
symmetric quadratic dict w/ name:	 {('y', 'x'): 1, ('x', 'y'): 1, ('y', 'y'): -1}
quadratic matrix:
 [[ 0  2]
 [ 0 -1]] 

symmetric quadratic matrix:
 [[ 0  1]
 [ 1 -1]] 

quadratic matrix as sparse matrix:
   (0, 1)	2
  (1, 1)	-1


In [36]:
#Adding/removing linear and quadratic constraints
#You can add linear constraints by setting name, linear expression, sense and right-hand-side value (rhs). You can use senses ‘EQ’, ‘LE’, and ‘GE’ as Docplex supports.

# Add linear constraints
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='==', rhs=3, name='lin_eq')
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='<=', rhs=3, name='lin_leq')
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='>=', rhs=3, name='lin_geq')
print(mod.export_as_lp_string())



QiskitOptimizationError: "Linear constraint's name already exists: lin_leq"

In [37]:
#You can access linear and quadratic terms of linear and quadratic constraints as in the same way as 
#the objective function.
lin_geq = mod.get_linear_constraint('lin_geq')
print('lin_geq:', lin_geq.linear.to_dict(use_name=True), lin_geq.sense, lin_geq.rhs)
quad_geq = mod.get_quadratic_constraint('quad_geq')
print('quad_geq:', quad_geq.linear.to_dict(use_name=True), quad_geq.quadratic.to_dict(use_name=True), quad_geq.sense, lin_geq.rhs)

lin_geq: {'x': 1.0, 'y': 2.0} ConstraintSense.GE 3


KeyError: 'quad_geq'

In [38]:
#You can also remove linear/quadratic constraints by remove_linear_constraint and remove_quadratic_constraint.

# Remove constraints
mod.remove_linear_constraint('lin_eq')
mod.remove_quadratic_constraint('quad_leq')
print(mod.export_as_lp_string())



KeyError: 'quad_leq'

In [39]:
#You can substitute some of variables with constants or other variables. More precicely, QuadraticProgram has a method substitute_variables(constants=..., variables=...) to deal with the following two cases. - x←c: when constants have a dictionary {x: c}. - x←cy: when variables have a dictionary {x: (y, c)}.
#Substituting Variables
sub = mod.substitute_variables(constants={'x': 0}, variables={'y': ('z', -1)})
print(sub.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: [ - 2 z^2 ]/2 + 3
Subject To
 lin_leq: - 2 z <= 3
 lin_geq: - 2 z >= 3

Bounds
 -1 <= z <= 1
End



In [40]:
#If the resulting problem is infeasible due to lower bounds or upper bounds, the methods returns the status Status.INFEASIBLE. We try to replace variable x with -1, but -1 is out of range of x (0 <= x <= 1). So, it returns Status.INFEASIBLE.
sub = mod.substitute_variables(constants={'x': -1})
print(sub.status)



Infeasible substitution for variable: x


QuadraticProgramStatus.INFEASIBLE


In [41]:
#You cannot substitute variables multiple times. The method raises an error in such a case.
from qiskit.optimization import QiskitOptimizationError
try:
    sub = mod.substitute_variables(constants={'x': -1}, variables={'y': ('x', 1)})
except QiskitOptimizationError as e:
    print('Error: {}'.format(e))



Error: 'Cannot substitute by variable that gets substituted itself: y <- x 1'
